In [1]:
# Suggested data science package imports
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

# TrendMiner package import
import trendminer
from trendminer.trendminer_client import TrendMinerClient

import os

token = os.environ["KERNEL_USER_TOKEN"]
serverUrl = os.environ["KERNEL_SERVER_URL"]

# Create TrendMiner API object
client = TrendMinerClient(token, serverUrl)

In [ ]:
import json
import requests
import time

auth_header = {'Authorization': f'Bearer {token}'}

In [ ]:
def prepare_search_conditions_calculations(search_conditions, define_calculations):

    search_conditions_list = []

    for condition in search_conditions:

        r = get_tag_details(condition[0])
        interpolationType = r.json()['interpolationType']

        tagname, condition, value = condition
        condition_dict = {
            "reference": {
                "name": tagname,
                "interpolationType": interpolationType,
                "shift": 0
            },
            "condition": condition,
            "values": [value]
        }
        search_conditions_list.append(condition_dict)


    search_calculations_list = []

    for calculation in define_calculations:

        r = get_tag_details(calculation[2])
        interpolationType = r.json()['interpolationType']
        tagId = r.json()['id']

        calculationName, calculationType, calculationTag = calculation
        calculation_dict = {
            "name": calculationName,
            "unit": "",
            "type": calculationType,
            "reference": {
                "id": tagId,
                "name": calculationTag,
                "interpolationType": interpolationType,
                "shift": 0
            }
        }
        search_calculations_list.append(calculation_dict)
        
    return search_conditions_list, search_calculations_list


def value_search_and_calculations(startDate, endDate, conditions, minDuration, operator, calculations):
    
    request_body= {
      "contextTimePeriod": {
        "startDate": startDate,
        "endDate": endDate
      },
      "definition": {
        "type": "VALUE_BASED_SEARCH",
        "queries": conditions,
        "parameters": {
          "minimumDuration": minDuration,
          "operator": operator
        },
        "calculations": calculations
      }
    }
    
    r = requests.post(url=f'{serverUrl}/compute/search-requests', json=request_body, headers=auth_header)
    
    return r.json()


def get_tag_details(name):
    
    params = {'tagName': name}
    
    r = requests.get(f'{serverUrl}/hps/api/tags/details', params=params, headers=auth_header)
        
    return r

    
def search_calcs_results_status(requestID):

    r = requests.get(url=f'{serverUrl}/compute/search-requests/{requestID}', headers=auth_header)
    
    return r.json()


def search_calcs_results(requestID, page, size):

    r = requests.get(url=f'{serverUrl}/compute/search-requests/{requestID}/results?page={page}&size={size}', headers=auth_header)
    
    return r.json()

In [ ]:
from datetime import datetime, timedelta
import pytz

timezone = 'America/Mexico_City'  # Time zone to use for date ranges

daysDelta = 10

startDate=(datetime.now()-timedelta(days=daysDelta)).strftime("%Y-%m-%dT%H:%M:%S.000Z")    # e.g. 2022-12-28T00:00:00.000Z
endDate=datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z")                                  # e.g. 2023-03-25T00:00:00.000Z

minDuration = 400

operator = "AND"

search_conditions = [
    ("BA:CONC.1", ">", 1),
    ("BA:TEMP.1", "<", 28)
]

define_calculations = [
    ("calculation name 1", "MEAN", "BA:TEMP.1"),
    ("calculation name 2", "MEAN", "BA:CONC.1")
]

requestSize = 600   #in seconds (minimum in 120s)

In [ ]:
search_conditions_list, search_calculations_list = prepare_search_conditions_calculations(search_conditions, define_calculations)

r_search_calcs = value_search_and_calculations(startDate, endDate, search_conditions_list, minDuration, operator, search_calculations_list)
requestID = r_search_calcs['id']

time.sleep(5)

r_search_calcs_results = search_calcs_results(requestID, 0, requestSize)

all_content = []
all_content.extend(r_search_calcs_results['content'])

# Loop through each page in the response
for i in range(1, r_search_calcs_results['page']['totalPages']-1):
    all_content.extend((search_calcs_results(requestID, i, requestSize))['content'])

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(all_content)

    # Normalize the dictionary column
df_normalized = pd.json_normalize(df['calculations'])

# Concatenate the normalized dataframe with the original dataframe
df = pd.concat([df, df_normalized], axis=1)

# Drop the original dictionary column
df.drop('calculations', axis=1, inplace=True)


df['start'] = pd.to_datetime(df['start'])
df['end'] = pd.to_datetime(df['end'])

current_time_zone = pytz.timezone(timezone)  # Replace with your desired time zone

df['start'] = df['start'].dt.tz_convert(current_time_zone)
df['end'] = df['end'].dt.tz_convert(current_time_zone)

# Display the dataframe
df